In [2]:
import os, zipfile, requests
import pandas as pd

symbol = 'BTCUSDT'
interval = '1d'
years = [2022, 2023]
url_base = 'https://data.binance.vision/data/spot/monthly/klines'

dfs = []

for y in years:
    for m in range(1, 13):
        fn = f"{symbol}-{interval}-{y}-{m:02d}"
        url = f"{url_base}/{symbol}/{interval}/{fn}.zip"
        r = requests.get(url)
        if r.status_code != 200: continue

        with open(f"{fn}.zip", 'wb') as f: f.write(r.content)
        with zipfile.ZipFile(f"{fn}.zip", 'r') as z: z.extractall()

        df = pd.read_csv(f"{fn}.csv", header=None)
        df.columns = ['timestamp','open','high','low','close','volume',
                      'close_time','quote_volume','trades',
                      'taker_base_vol','taker_quote_vol','ignore']
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        dfs.append(df)

        os.remove(f"{fn}.zip")
        os.remove(f"{fn}.csv")

result = pd.concat(dfs, ignore_index=True)
result.to_csv(f"{symbol}_{interval}_{years[0]}_{years[-1]}.csv", index=False)
